In [85]:
import spacy
from tqdm.notebook import tqdm
import pandas as pd

In [86]:
nlp = spacy.load("fr_core_news_sm")
brute_sentences_df = pd.read_csv('../data/dataset/brute_sentences_2.csv')
train_station_df = pd.read_csv('../data/liste-des-gares.csv', sep=';')

In [87]:
def apply_iob_to_sentence(doc_sentence, clean_df, departure, arrival):
    for word in [token_word.text for token_word in doc_sentence]:
        if (word in ["DEPARTURE", "ARRIVAL", "NONE", "INTERMEDIARY"]):
            plus_sub_words = str(train_station_df.sample()['LIBELLE'].values[0])
            plus_sub_words = plus_sub_words.split()
            sub_words = (str(departure).split() if word == "DEPARTURE" else str(arrival).split())

            for sub_word_idx, sub_word in enumerate((sub_words if word in ["DEPARTURE", "ARRIVAL"] else plus_sub_words)):
                if (sub_word_idx == 0) :
                    clean_df.append(
                        {
                            "word": sub_word,
                            "status": "B"
                        }
                    )
                    continue
                clean_df.append(
                    {
                        "word": sub_word,
                        "status": "I"
                    }
                )
            continue
        clean_df.append(
            {
                "word": word,
                "status": "O"
            }
        )

In [88]:
clean_df = [{"word": None, "status": None}]

with tqdm(total= 1000 * len(brute_sentences_df), unit="train station") as pbar:
    for idx, sentence in enumerate(brute_sentences_df["sentence"]):
        for i in range(1000):
            departure = str(train_station_df.sample()["LIBELLE"].values[0])
            arrival = str(train_station_df.sample()["LIBELLE"].values[0])

            while(arrival == departure):
                arrival = str(train_station_df.sample()["LIBELLE"].values[0])

            doc_sentence = nlp(sentence)

            apply_iob_to_sentence(
                doc_sentence,
                clean_df,
                departure,
                arrival
            )

            clean_df.append(
                {
                    "word": None,
                    "status": None
                }               
            )
            pbar.update(1)

clean_df = pd.DataFrame(clean_df)

  0%|          | 0/151000 [00:00<?, ?train station/s]

In [96]:
new_df = pd.DataFrame(columns=["sub_sentence", "status"])

with tqdm(
    # total=clean_df.shape[0],
    total=50000,
    unit="sentence") as pbar:
    for idx, row in clean_df.iloc[1:50000].iterrows():
        sub_sentence = f"{clean_df.iloc[idx - 1].word}[{clean_df.iloc[idx - 1].status}] "
        sub_sentence += f"{clean_df.iloc[idx].word} {clean_df.iloc[idx + 1].word}"
        
        new_row = {
            "sub_sentence": sub_sentence,
            "status": clean_df.iloc[idx].status
        }
        new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
        pbar.update(1)

  0%|          | 0/50000 [00:00<?, ?sentence/s]

In [100]:
new_df_shuffled = new_df.sample(frac=1)
train_df = new_df_shuffled[:int(len(new_df_shuffled)*0.7)]
valid_df = new_df_shuffled[int(len(new_df_shuffled)*0.7):int(len(new_df_shuffled)*0.85)]
test_df = new_df_shuffled[int(len(new_df_shuffled)*0.85):]

In [102]:
train_df.to_csv('../data/dataset/ner_train.csv', index=False)
valid_df.to_csv('../data/dataset/ner_valid.csv', index=False)
test_df.to_csv('../data/dataset/ner_test.csv', index=False)